In [ ]:
import numpy as np
import glob
from collections import Counter
li=[]
# the folder 20_newsgroups is in the same path as the jupyter notebook
for infile in glob.glob('data/traindata/*/*'):
    
    try:
        review_file = open(infile,'r').read()
            
    except UnicodeDecodeError:
            print('exception')
    li.append(review_file.split(" "))

In [ ]:
len(li)

In [ ]:
def flatten(input):
    new_list = []
    for i in input:
        for j in i:
            new_list.append(j)
    return new_list


In [ ]:
li=flatten(li)       # to make a 1d list out of a 2d list containing words via split(' ') from training documents

In [ ]:
stop=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [ ]:
dic=Counter(li)
wordlist=[]
for i in dic:
    wordlist.append([dic[i],i])                    # making a 2d list with unique words with their counts
l=len(wordlist)
print('len--',l)
i=0
while i < l:
    if wordlist[i][0]<5 or wordlist[i][1]=='':     # removing empty strings and words with count less than  5 
        print(i)
        wordlist.pop(i)
        i-=1
        l-=1
    i+=1
i=0
while i < l:
    if wordlist[i][1].lower() in stop:           # removing stop words
        print(i)
        wordlist.pop(i)
        i-=1
        l-=1
    i+=1
i=0
while i < l:
    if wordlist[i][1].isalpha()==False:          # selecting srings that contain only alphabets to get filtered words
        print(i)
        wordlist.pop(i)
        i-=1
        l-=1
    i+=1

In [ ]:
len(wordlist)

In [ ]:
# 2d list with unique filtered words with count >=5
wordlist         

In [ ]:
iterlist=sorted(wordlist,reverse=True)     

In [ ]:
print(iterlist)    # sorted 2d list with unique words to select top words for the vocabulary

In [ ]:
vocabulary=[]
for i in range(6000):                    # forming the vocabulary list by selecting top 6000 words via count
    vocabulary.append(iterlist[i][1])

In [ ]:
vocabulary               # vocabulary list

In [ ]:
import csv
import numpy as np
import pandas as pd

In [ ]:
vocab=np.asarray(vocabulary)

df=pd.DataFrame(vocab)                         #saving the vocabulary list as .csv file for further use such that data
                                                # loading doesnt have to be done again and again
df.to_csv('vocab.csv',sep=',',header=False,index=False)

In [ ]:
import time
with open('vocab.csv', 'r') as f:
    reader = csv.reader(f)
    vocab = list(reader)
vocab=flatten(vocab)                        # reding the vocab.csv file to get the vocabulary list
print(len(vocab))
col=[]
col=vocab[:]
col.append('Target')
traindf=pd.DataFrame(index= range(20) , columns=np.array(col))    # forming the training dataframe containing the word
print(len(col),len(vocab))                                        # count for every word in vocabulary in a particular 
                                                                  # class.

In [ ]:
start=time.time()
for i in range(20):
    li=[]
    trainlist=[]
    for infile in glob.glob('data/traindata/'+str(i)+'/*'):  # path: ~/i/* for class i
        try:
            review_file = open(infile,'r').read()       #reading training files 
            
        except UnicodeDecodeError:
            print('exception')
            
        li.append(review_file.split(" "))                    #TRAINING 
        
    li=flatten(li)
    wordict=Counter(li)
    for word in vocab:
        if word in wordict:                             # counting the words present in vocabulary in the files 
            trainlist.append(wordict[word])             # for a particular class
        else:
            trainlist.append(0)
    trainlist.append(i)
    traindf.iloc[i,:]=np.asarray(trainlist)            # appending to the training dataframe
    print(i)
end=time.time()
print(end-start)

In [ ]:
traindf.shape

In [ ]:
testdf=pd.DataFrame(index=range(4000),columns=np.array(vocab))
testdf.fillna(0,inplace=True)           # creating a testing dataframe for 4000 files
testdf.shape

In [ ]:
i=0
for infile in glob.glob('testfilenamed/*'): #path where all the test files are stores together
    testli=[]
    countlist=[]
    
    try:
        review_file = open(infile,'r').read()           #reading tesfiles
            
    except UnicodeDecodeError:
            print('exception agya')
            
    
    testli.append(review_file.split(" "))          # TESTING
    print('i---',i)
    testli=flatten(testli)

    wordcount=Counter(testli)
        
    for word in vocab:
        if word in wordcount:
            countlist.append(wordcount[word])  # appending to  a list  storing count of words from the vocabulary in
        else:                                   # single test file.
            countlist.append(0)
    testdf.iloc[i,:]=np.asarray(countlist)     # forming the testing dataframe containing count of vocabulary words in 
                                              # a single test file for 4000 files.
    
    i+=1

In [ ]:
testdf.shape

# Inbuilt Multinomial Naive Bayes for Text Classification

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()                              #initialising the inbuilt multinomial naive bayes classifier
clf.fit(traindf.iloc[:,0:-1],traindf.iloc[:,-1])

In [ ]:
ylist=[]
for infile in glob.glob('testfilenamed/*'):
    li=infile.split('/')
    var=li[-1].split('-')            # getting the actual classes of the testfiles 
    ylist.append(var[0])             # from their names(mmaped to a range - 0 to 19 for convenience)
    
ylist = list(map(int, ylist))
ylist
ydf=pd.DataFrame(np.array(ylist).reshape(4000,1),columns=['y'])

## Score with inbulit naive bayes classifier

In [ ]:
clf.score(testdf,ydf)                  # testing with the inbuilt naive bayes classifier 

# Self Implimented Multinomial Naive Bayes for Text Classification

In [ ]:
def probability(testdf,vocab,df):
    
    probab_list=[]
    for k in range(4000):  # for 4000 test files 
        
        print(k)
        plist=[]
        
        for i in range(20):     # checking for every class 
            
            out=0
            a=time.time()
            
            for j in range(6000):       # for every word in the vocabulary
                
                out+=np.log((testdf.iloc[k,j]+1))-np.log(sum(traindf.iloc[i,:])+6000)  # calculating word log probabilities
                
            b=time.time()
            print('ab',b-a)
            
            prob=out+np.log(1/20)  #adding log probability of a class
            plist.append(prob)
              
        print(plist)
        maxindex=plist.index(max(plist))  # getting the index of the class with the maximum probability
        print('pred',maxindex)
        probab_list.append(maxindex)
    
    
    print('prob--',prob)
    return probab_list        # returning the list with predicted classes  for  4000 test files
        
        
    

In [ ]:
class_prob=1/20
prediction=[]
st=time.time()                                                  # SELF IMPLIMENTED NAIVE BAYES CLASSIFIER
problist=probability(testdf,vocab,traindf)                   # being called for predictions from the testing dataframe
print(problist)
en=time.time()
print('time taken - ',en-st)

In [ ]:
print(ylist)

In [ ]:
def score(prediction,ylist):
    a=np.array(prediction)
    b=np.array(ylist)              # score function for the self implimented naive bayes classifier
    c=list(a-b)
    total=len(c)
    r=c.count(0)
    print(r/total)

## Score with self implimented naive bayes classifier

In [ ]:
score(prediction,ylist)